In [10]:
import os
os.listdir("/kaggle/input")

['spacenet-an-optimally-distributed-astronomy-data']

In [11]:
import os

base_path = "/kaggle/input/spacenet-an-optimally-distributed-astronomy-data/SpaceNet.FLARE.imam_alam"
os.listdir(base_path)

['planet',
 'galaxy',
 'black hole',
 'asteroid',
 'comet',
 'star',
 'constellation',
 'nebula']

In [26]:
NUM_CLASSES = 8


In [50]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])
dataset = datasets.ImageFolder(root=base_path, transform=transform)

data_loader = DataLoader(
    dataset,
    batch_size=32,
    shuffle=False
)


***CNN FROM SCRATCH***

There will be three things hapenning 
1. Convolution
2. Pooling
3. Layers
   

In [52]:
import torch
import torch.nn as nn
import torch.optim as optim

In [53]:
class meraCNN(nn.Module):
    def __init__(self,num_classes):
        super(meraCNN,self).__init__()
        

#convolution + pooling
        self.conv1 = nn.Conv2d(
            in_channels=3,   #RGB 3
            out_channels = 16,   
            kernel_size = 3,
            padding=1
        )

        self.pool = nn.MaxPool2d(kernel_size=2,stride=2)
        #height and width will be half

        #now i will add 2nd layer
        self.conv2 = nn.Conv2d(16,32,kernel_size=3,padding=1)
        #input 16 and output will 32 

        #compressing  image info to 128 neurons

        self.fc1 = nn.Linear(32*16*16, 128)
        self.fc2 = nn.Linear(128,num_classes)


    def forward(self,x):

        x = torch.relu(self.conv1(x))

        x=self.pool(x)
        x=torch.relu(self.conv2(x))
        x=self.pool(x)
        x = x.view(x.size(0), -1)

        x = torch.relu(self.fc1(x))
        x = self.fc2(x)

        return x



In [56]:
model = meraCNN(NUM_CLASSES)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


to evalute (copied)

In [62]:
from torchvision.datasets import ImageFolder
from PIL import Image

class SafeImageFolder(ImageFolder):
    def __getitem__(self, index):
        path, target = self.samples[index]
        try:
            image = Image.open(path).convert("RGB")
            if self.transform is not None:
                image = self.transform(image)
            return image, target
        except Exception:
            # skip corrupted image by returning another one
            return self.__getitem__((index + 1) % len(self.samples))

base_path = "/kaggle/input/spacenet-an-optimally-distributed-astronomy-data/SpaceNet.FLARE.imam_alam"

dataset = SafeImageFolder(root=base_path, transform=transform)

print("Total images:", len(dataset))
print("Classes:", dataset.classes)




Total images: 12824
Classes: ['asteroid', 'black hole', 'comet', 'constellation', 'galaxy', 'nebula', 'planet', 'star']


In [66]:
from torch.utils.data import random_split

from torch.utils.data import Subset
import random

MAX_TEST_IMAGES = 800  # tune this (300–1000)

indices = random.sample(range(len(dataset)), MAX_TEST_IMAGES)
test_dataset = Subset(dataset, indices)



In [68]:
from torch.utils.data import DataLoader

test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)


In [69]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


Test Accuracy: 21.25%
